In [2]:
import os 
import numpy as np
import pandas as pd
%reload_ext autoreload
%autoreload 2

In [3]:
lt_logs_df = pd.read_csv('./logs/long_term_forecast.csv', index_col = 0)
from types import SimpleNamespace
args = SimpleNamespace(**lt_logs_df.loc[21].to_dict())

### ETT

In [4]:
from data_provider.data_loader import Dataset_ETT_minute
from data_provider.data_factory import data_provider

ettds, ettdl = data_provider(args = args, flag = 'train')

train 34273


In [5]:
ettds.data_x.shape

(34560, 7)

In [6]:
ettds.__len__(), ettdl.__len__(), ettdl.batch_size

(34273, 536, 64)

In [7]:
print(len(ettds.__getitem__(0)))
ettds.__getitem__(0)[0].shape

4


(96, 7)

### USC

In [15]:
sequence_len = 48
target_len = 192
dim = 32
args.seq_len = sequence_len
args.pred_len = target_len
args.root_path = './dataset/USC/'
args.data = 'USC'
args.input_features = 'V'
args.use_action_progress = True
args.use_offense = True
args.enc_in, args.dec_in, args.c_out = dim, dim , dim
ind_coords = int(args.use_action_progress) + int(args.use_offense)

uscds, uscdl = data_provider(args= args, flag = 'train')
np.isnan(uscds.data_x).any(), np.isnan(uscds.data_y).any(), uscds.data_x.shape , uscds.data_x[:,:,ind_coords:].min(), uscds.data_x[:,:,ind_coords:].max()

train 29509


(False, False, (29509, 48, 32), -263.25995, 219.6671)

In [9]:
uscds.data_x.shape, len(uscds.data_x),  uscdl.__len__(), uscdl.batch_size

((24542, 64, 32), 24542, 384, 64)

In [17]:
x_coords = [False]*3 + [i %4 == 3 for i in range(3, 63)]
y_coords = [False]*3 + [i %4 == 0 for i in range(3, 63)]
vx_coords = [False]*3 + [i %4 == 1 for i in range(3, 63)]
vy_coords = [False]*3 + [i %4 == 2 for i in range(3, 63)]
stats = []
for f in os.listdir(args.root_path):
    print(f)
    tmp = np.load(os.path.join(args.root_path, f))
    x_min, x_max, y_min, y_max = tmp[:,:,x_coords].min(), tmp[:,:,x_coords].max(), tmp[:,:,y_coords].min(), tmp[:,:,y_coords].max()
    vx_min, vx_max, vy_min, vy_max = tmp[:,:,vx_coords].min(), tmp[:,:,vx_coords].max(), tmp[:,:,vy_coords].min(), tmp[:,:,vy_coords].max()
    g_min, g_max = tmp[:,:,1:].min(), tmp[:,:,1:].max()
    stats.append({'f':f, 'shape':tmp.shape, 'min':g_min, 'max':g_max, 
                  'x_min':x_min, 'x_max':x_max, 'y_min':y_min, 'y_max':y_max, 
                  'vx_min':vx_min, 'vx_max':vx_max, 'vy_min':vy_min, 'vy_max':vy_max})
pd.DataFrame(stats)

PVTO_seq48_tar48_X.npy
PVTO_seq32_tar32_y.npy
PVTO_seq24_tar24_X.npy
PVTO_seq24_tar24_y.npy
PVTO_seq32_tar32_X.npy
PVTO_seq16_tar16_X.npy
PVTO_seq64_tar64_y.npy
PVTO_seq48_tar48_y.npy
PVTO_seq16_tar16_y.npy
PVTO_seq64_tar64_X.npy


,f,shape,min,max,x_min,x_max,y_min,y_max,vx_min,vx_max,vy_min,vy_max
0,PVTO_seq48_tar48_X.npy,"(49183, 48, 63)",-4.666577e+06,129.939774,-29.616024,129.939774,-19.945639,89.974266,-55.459255,10.666189,-27.167757,43.293716
1,PVTO_seq32_tar32_y.npy,"(58181, 32, 63)",-5.708128e+01,129.964691,-29.653769,129.964691,-19.986343,89.974266,-55.459255,58.255981,-27.167757,43.293716
2,PVTO_seq24_tar24_X.npy,"(63762, 24, 63)",-4.666577e+06,129.939774,-29.616024,129.939774,-19.952444,89.974266,-55.459255,24.223558,-27.167757,43.293716
3,PVTO_seq24_tar24_y.npy,"(63762, 24, 63)",-4.666577e+06,129.964691,-29.653769,129.964691,-19.995951,89.974266,-55.459255,58.255981,-27.167757,43.293716
4,PVTO_seq32_tar32_X.npy,"(58181, 32, 63)",-5.545926e+01,129.939774,-29.616024,129.939774,-19.945639,89.974266,-55.459255,10.666189,-27.167757,43.293716
5,PVTO_seq16_tar16_X.npy,"(69435, 16, 63)",-4.666577e+06,129.945679,-29.616024,129.945679,-19.952444,89.974266,-59.726681,24.223558,-27.167757,44.986813
6,PVTO_seq64_tar64_y.npy,"(40904, 64, 63)",-4.666577e+06,129.964691,-29.653769,129.964691,-18.887793,89.974266,-55.459255,13.723037,-27.167757,16.384754
7,PVTO_seq48_tar48_y.npy,"(49183, 48, 63)",-4.666577e+06,129.964691,-29.653769,129.964691,-19.986343,89.974266,-55.459255,58.255981,-27.167757,43.293716
8,PVTO_seq16_tar16_y.npy,"(69435, 16, 63)",-4.666577e+06,129.964691,-29.653769,129.964691,-19.995951,89.974266,-59.726681,58.255981,-27.167757,44.986813
9,PVTO_seq64_tar64_X.npy,"(40904, 64, 63)",-4.666577e+06,129.939774,-29.616024,129.939774,-18.236652,89.974266,-55.459255,10.195284,-27.167757,6.738795


In [16]:
from models.LSTransformer import Model
from utils.model_wrapper import ModelWrapper
model = ModelWrapper(Model(configs=args), args)
model.summary()

Shape step 1 : torch.Size([1, 48, 32]) - input
Shape step 2 : torch.Size([1, 48, 256]) - post LSTM
Shape step 3 : torch.Size([1, 48, 512]) - post projection
Shape step 4 : torch.Size([1, 48, 512]) - post Transformer
Shape step 5 : torch.Size([1, 48, 256]) - post Linear
Shape step 6 : torch.Size([1, 48, 192]) - post output layer
Shape step 7 : torch.Size([1, 192, 32]) - post permute & truncate
Model quick summary
Layer name                    |          Nb units|      Nb tr params|  Nb non tr params
LSTM_0                        |               256|           296 960|                 0
Linear_0                      |               256|             8 448|                 0
LayerNorm_0                   |                 -|               512|                 0
Linear_1                      |               512|           131 584|                 0
TransformerEncoderLayer_0     |               512|         3 152 384|                 0
TransformerEncoderLayer_1     |               512|      